In [ ]:
import tensorflow as tf
import keras

import sys
sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import dataset
import numpy as np

import hls4ml
import plotting
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# load data

In [ ]:
BATCH_SIZE = 64
EPOCHS = 100

train_images_rgx = "../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"

train_dataset, val_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE
)

# load model

In [ ]:
model_path = "/home/webphy/Desktop/dnn_processor/neural_network/automl/saved_models/striped_models/295/295.h5"

org_model = keras.models.load_model(model_path, compile=True)
org_model.evaluate(val_dataset)


# creating HLS4ML model

In [ ]:



model = org_model
config = hls4ml.utils.config_from_keras_model(model, granularity='model')
print("-----------------------------------")
print("Configuration")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='model_1/hls4ml_prj', part='xcu250-figd2104-2L-e'
)

In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
hls_model.compile()
X_test = np.ascontiguousarray(val_dataset)
y_hls = hls_model.predict(X_test)